In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from util.s3dis import S3DIS
import k3d
from matplotlib import cm, colors
import torch
import numpy as np

In [2]:
def visualize_pointcloud(point_cloud, point_size=0.05, colors=None, flip_axes=False, name='point_cloud'):
    plot = k3d.plot(name=name, grid_visible=False)
    if flip_axes:
        point_cloud[:, 2] = point_cloud[:, 2] * -1
        point_cloud[:, [0, 1, 2]] = point_cloud[:, [0, 2, 1]]
    plt_points = k3d.points(positions=point_cloud.astype(np.float32), point_size=point_size, colors=colors if colors is not None else [], color=0xd0d0d0)
    plot += plt_points
    plt_points.shader = '3d'
    return plot.display()


In [12]:
from model.pointtransformer.pointtransformer_seg import pointtransformer_seg_repro as Model
import collections
classes = 13
model = Model(c=6,k=classes).cuda()
print(model)

# load the checkpoint
model_path = "./exp/s3dis/pointtransformer_repro/model/model_best.pth"
checkpoint = torch.load(model_path)
state_dict = checkpoint["state_dict"]
new_state_dict = collections.OrderedDict()
for k, v in state_dict.items():
    name = k[7:]
    new_state_dict[name] = v
model.load_state_dict(new_state_dict, strict=True)

PointTransformerSeg(
  (enc1): Sequential(
    (0): TransitionDown(
      (linear): Linear(in_features=6, out_features=32, bias=False)
      (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): PointTransformerBlock(
      (linear1): Linear(in_features=32, out_features=32, bias=False)
      (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (transformer2): PointTransformerLayer(
        (linear_q): Linear(in_features=32, out_features=32, bias=True)
        (linear_k): Linear(in_features=32, out_features=32, bias=True)
        (linear_v): Linear(in_features=32, out_features=32, bias=True)
        (linear_p): Sequential(
          (0): Linear(in_features=3, out_features=3, bias=True)
          (1): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Linear(in_features=3, out_features=32, bias

<All keys matched successfully>

In [13]:
import os
data_root = "dataset/s3dis/trainval_fullarea"
model.eval()
data_list = sorted(os.listdir(data_root))
data_list = [item[:-4] for item in data_list if 'Area_{}'.format(5) in item]


In [14]:
from util.voxelize import voxelize
def data_load(data_name):
    voxel_size = 0.04
    data_path = os.path.join("dataset/s3dis/trainval_fullarea", data_name + '.npy')
    data = np.load(data_path)  # xyzrgbl, N*7
    coord, feat, label = data[:, :3], data[:, 3:6], data[:, 6]

    idx_data = []
    if voxel_size:
        coord_min = np.min(coord, 0)
        coord -= coord_min
        idx_sort, count = voxelize(coord, voxel_size, mode=1)
        for i in range(count.max()):
            idx_select = np.cumsum(np.insert(count, 0, 0)[0:-1]) + i % count
            idx_part = idx_sort[idx_select]
            idx_data.append(idx_part)
    else:
        idx_data.append(np.arange(label.shape[0]))
    return coord, feat, label, idx_data

In [15]:
def input_normalize(coord, feat):
    coord_min = np.min(coord, 0)
    coord -= coord_min
    feat = feat / 255.
    return coord, feat

In [16]:
import pickle
def infer(model, item, version, calculate=False):
    batch_size_test = 10
    voxel_max = 80000
    save_folder = "/home/nazmicancalik/workspace/point-transformer"
    model.eval()
    
    coord, feat = None, None
    pred_save, label_save = [], []
    pred_save_path = os.path.join(save_folder, '{}_{}_pred.npy'.format(item, version))
    label_save_path = os.path.join(save_folder, '{}_{}_label.npy'.format(item, version))
    if os.path.isfile(pred_save_path) and os.path.isfile(label_save_path) and not calculate:
        print('loaded pred and label')
        pred, label = np.load(pred_save_path), np.load(label_save_path)
    else:
        coord, feat, label, idx_data = data_load(item)
        pred = torch.zeros((label.size, 13)).cuda()
        idx_size = len(idx_data)
        idx_list, coord_list, feat_list, offset_list  = [], [], [], []
        for i in range(idx_size):
            print('{}/{}'.format(i + 1, idx_size))
            idx_part = idx_data[i]
            coord_part, feat_part = coord[idx_part], feat[idx_part]
            if voxel_max and coord_part.shape[0] > voxel_max:
                coord_p, idx_uni, cnt = np.random.rand(coord_part.shape[0]) * 1e-3, np.array([]), 0
                while idx_uni.size != idx_part.shape[0]:
                    init_idx = np.argmin(coord_p)
                    dist = np.sum(np.power(coord_part - coord_part[init_idx], 2), 1)
                    idx_crop = np.argsort(dist)[:voxel_max]
                    coord_sub, feat_sub, idx_sub = coord_part[idx_crop], feat_part[idx_crop], idx_part[idx_crop]
                    dist = dist[idx_crop]
                    delta = np.square(1 - dist / np.max(dist))
                    coord_p[idx_crop] += delta
                    coord_sub, feat_sub = input_normalize(coord_sub, feat_sub)
                    idx_list.append(idx_sub), coord_list.append(coord_sub), feat_list.append(feat_sub), offset_list.append(idx_sub.size)
                    idx_uni = np.unique(np.concatenate((idx_uni, idx_sub)))
            else:
                coord_part, feat_part = input_normalize(coord_part, feat_part)
                idx_list.append(idx_part), coord_list.append(coord_part), feat_list.append(feat_part), offset_list.append(idx_part.size)
        batch_num = int(np.ceil(len(idx_list) / batch_size_test))
        for i in range(batch_num):
            s_i, e_i = i * batch_size_test, min((i + 1) * batch_size_test, len(idx_list))
            idx_part, coord_part, feat_part, offset_part = idx_list[s_i:e_i], coord_list[s_i:e_i], feat_list[s_i:e_i], offset_list[s_i:e_i]
            idx_part = np.concatenate(idx_part)
            coord_part = torch.FloatTensor(np.concatenate(coord_part)).cuda(non_blocking=True)
            feat_part = torch.FloatTensor(np.concatenate(feat_part)).cuda(non_blocking=True)
            offset_part = torch.IntTensor(np.cumsum(offset_part)).cuda(non_blocking=True)
            with torch.no_grad():
                pred_part = model([coord_part, feat_part, offset_part])  # (n, k)
            torch.cuda.empty_cache()
            pred[idx_part, :] += pred_part
            print('Test: {}/{}, {}/{}, {}/{}'.format(1, 1, e_i, len(idx_list), voxel_max, idx_part.shape[0]))
        pred = pred.max(1)[1].data.cpu().numpy()

    pred_save.append(pred); label_save.append(label)
    np.save(pred_save_path, pred); np.save(label_save_path, label)
    return coord, feat, pred, label

In [17]:
def plot(coord, prediction):
    point_labels = (prediction - min(prediction)) / (max(prediction) - min(prediction))
    point_colors = cm.get_cmap('hsv')(point_labels)[:, :3]
    point_colors = np.sum((point_colors * 255).astype(int) * [255*255, 255, 1], axis=1)
    visualize_pointcloud(coord, colors=point_colors, point_size=0.025, flip_axes=True)

In [18]:
# Load the item and test
test_item = "Area_5_office_2"
version = "w"
coord, feat, pred, label = infer(model, test_item, version, True)
print(pred.size)

1/55
2/55
3/55
4/55
5/55
6/55
7/55
8/55
9/55
10/55
11/55
12/55
13/55
14/55
15/55
16/55
17/55
18/55
19/55
20/55
21/55
22/55
23/55
24/55
25/55
26/55
27/55
28/55
29/55
30/55
31/55
32/55
33/55
34/55
35/55
36/55
37/55
38/55
39/55
40/55
41/55
42/55
43/55
44/55
45/55
46/55
47/55
48/55
49/55
50/55
51/55
52/55
53/55
54/55
55/55
Test: 1/1, 10/55, 80000/595090
Test: 1/1, 20/55, 80000/595090
Test: 1/1, 30/55, 80000/595090
Test: 1/1, 40/55, 80000/595090
Test: 1/1, 50/55, 80000/595090
Test: 1/1, 55/55, 80000/297545
867248


In [20]:
plot(coord,pred)

Output()